In [1]:
import sys
sys.path.append('src')
import numpy as np
import tensorflow as tf
from bridges import Model

In [2]:
m = Model()

Loading checkpoint checkpoint/run1/model-310052
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-310052


In [3]:
m.gen('blah blah')

['blah blah auxamer ». (']

In [41]:
def print_lists(ll):
    if not isinstance(ll[0],(list, tuple, np.ndarray)):
        print(ll)
    else:
        print(*ll, sep='\n')    

In [331]:
def normalize(logits, verbose=False):
    mu = np.mean(logits, axis=-1, keepdims=True)
    lm = logits - mu
    le = np.exp(lm)
    logprobs = le / np.sum(le, axis=-1, keepdims=True)
    if verbose:
        return logprobs, mu, lm, le
    return logprobs

In [338]:
logprobs, mu, lm, le = normalize(logits, verbose=True)

In [320]:
np.mean(logits[0,0])

7.638048

In [321]:
mu[0,0]

array([7.638048], dtype=float32)

In [327]:
lm[0,0,0] == logits[0,0,0] - mu[0,0]

array([ True])

In [328]:
np.exp(lm[0,0,0]) == le[0,0,0]

True

In [334]:
np.sum(le, axis=-1, keepdims=True).shape

(4, 12, 1)

In [337]:
np.sum(le, axis=-1, keepdims=True)[0,0]

array([14673772.], dtype=float32)

In [339]:
logprobs[0,0,0] == le[0,0,0] / np.sum(le, axis=-1, keepdims=True)[0,0, 0]

True

In [305]:
def get_perplexity(model, sentences=["\n"], batch_size=None, verbose=False):
    # save current batch_size
    batch_size_bak=None
    if batch_size is not None and self.batch_size != batch_size:
        batch_size_bak = self.batch_size
    model.check_batch_size(batch_size)
    if isinstance(sentences, str):
        sentences = [sentences]
    tkns = model.pad_sequences(sentences)
    print('sequences:')
    print_lists(tkns)
        
    logits = model.get_logits(context_tokens=tkns)[:, :-1, :] # None to keep dims intact
    # don't take the last one (predicting the token after our sentence)
    
    print("logits, shape:", logits.shape)
    
    # normalization between 0 and 1
    # most importantly: make all 
    logprobs = normalize(logits)
    print("logprobs, shape:", logprobs.shape)
    
#     return logprobs, tkns
#     print(tkns.shape)
#     for i, token in enumerate(tkns[:, :-1]):
#         print(i, token)

#     scores = np.nan_to_num(
#         [logprobs[:, i, token] for i, token in enumerate(tkns[:-1])]
#     )    
    scores = np.nan_to_num(
        [
            [logprobs[b, i, token] 
             for b in range(len(tkns))             
             for i, token in enumerate(tkns[:, :-1])]            
        ]
    )
    print("scores, shape:", scores.shape)
    perplexities = 2 ** (-np.mean(np.log2(scores), axis=-1))
#     # restoring batch size 
#     if batch_size_bak: model.check_batch_size(batch_size_bak)    
#     return perplexities

In [284]:
snt = [
    "Ah, nous voilà bien",
    "Je me baladais dans le gris sans fond",
    "LE COMTE. Quel déshonneur !",
    "Ah."
]

In [285]:
get_perplexity(m, snt)

sequences:
[  220   220   220 10910    11   299   516  7608   346 24247   275  2013]
[40932   502  3652  4763   271   288   504   443  1036   271 38078 16245]
[  220  2538  9440  9328    13  2264   417 39073  1477 47476   333  5145]
[  220   220   220   220   220   220   220   220   220   220 10910    13]
logits, shape: (4, 11, 50257)
logprobs, shape: (4, 11, 50257)
[[  220   220   220 10910    11   299   516  7608   346 24247   275]
 [40932   502  3652  4763   271   288   504   443  1036   271 38078]
 [  220  2538  9440  9328    13  2264   417 39073  1477 47476   333]
 [  220   220   220   220   220   220   220   220   220   220 10910]]
scores, shape: (1, 16, 11)


In [286]:
padded = m.pad_sequences(snt)
logits = m.get_logits(padded)

In [287]:
logits.shape

(4, 12, 50257)

In [340]:
logprobs = normalize(logits)

In [341]:
logprobs.shape

(4, 12, 50257)

In [343]:
scores = np.nan_to_num([[logprobs[j, i, token] 
 for i, token in enumerate(batch[:-1])]
 for j, batch in enumerate(padded)])

In [344]:
perplexities = 2 ** (-np.mean(np.log2(scores), axis=-1))

In [347]:
print(*perplexities, sep='\n')

77367.34
188906.8
2907181.0
2.7888317


In [251]:
logits[0]

array([[14.289879 ,  9.88566  , 10.730696 , ...,  5.7731833,  6.458465 ,
         5.841981 ],
       [ 3.1374109,  1.3310282,  4.4881535, ...,  3.720184 ,  3.8290942,
         3.7567344],
       [ 3.4757946,  0.8460077,  4.568767 , ...,  3.2985313,  3.3040564,
         3.3131304],
       ...,
       [ 2.0084343, -8.294467 , -6.12812  , ..., -9.5075   , -9.178121 ,
        -9.197476 ],
       [ 7.3087406,  6.4965568,  6.595199 , ...,  1.4063548,  2.3194256,
         1.880352 ],
       [ 3.4282572, -8.777307 , -6.4584146, ..., -9.818342 , -9.568864 ,
        -9.863892 ]], dtype=float32)

In [247]:
padded[0,:,None]

array([[  220],
       [  220],
       [  220],
       [10910],
       [   11],
       [  299],
       [  516],
       [ 7608],
       [  346],
       [24247],
       [  275],
       [ 2013]], dtype=int32)

In [ ]:
indices = [(b, i, tkn) for  in ]

In [167]:
list(zip(np.arange(len(padded[0])),padded[0]))

[(0, 220),
 (1, 220),
 (2, 220),
 (3, 10910),
 (4, 11),
 (5, 299),
 (6, 516),
 (7, 7608),
 (8, 346),
 (9, 24247),
 (10, 275),
 (11, 2013)]

In [238]:
for j, (b, tkns) in enumerate(zip(logits, padded)):
    for i, tkn in enumerate(tkns):
        print(f"{b[i,tkn]:>6.2f}", end=' ')
    print()

 22.68  18.83  18.52  -0.17  -7.89  -0.47  -9.74  -3.58   3.29  -7.50   0.54  -6.90 
  5.66   4.03   4.06 -11.92 -11.42   0.89  -4.92  -2.19   0.91  -5.07  -4.36  -2.13 
 22.68   1.53   7.14   8.36   0.53   1.24  -1.31  -0.13   0.08 -10.10  -5.05   2.72 
 22.68  18.83  18.52  18.08  19.06  20.29  21.18  21.54  21.77  21.78   1.73   0.93 


In [236]:
for j, (b, tkns) in enumerate(zip(logits, padded)):
    print('batch:', j)
    print('tokens:\t', end='')
    for tkn in tkns:
        print(f"{tkn:>6} | ", end='')
    print()
    print('logits:\t', end='')
    for i, tkn in enumerate(tkns):
        print(f"{b[i,tkn]:>6.2f} |", end=' ')
    print()

batch: 0
tokens:	   220 |    220 |    220 |  10910 |     11 |    299 |    516 |   7608 |    346 |  24247 |    275 |   2013 | 
logits:	 22.68 |  18.83 |  18.52 |  -0.17 |  -7.89 |  -0.47 |  -9.74 |  -3.58 |   3.29 |  -7.50 |   0.54 |  -6.90 | 
batch: 1
tokens:	 40932 |    502 |   3652 |   4763 |    271 |    288 |    504 |    443 |   1036 |    271 |  38078 |  16245 | 
logits:	  5.66 |   4.03 |   4.06 | -11.92 | -11.42 |   0.89 |  -4.92 |  -2.19 |   0.91 |  -5.07 |  -4.36 |  -2.13 | 
batch: 2
tokens:	   220 |   2538 |   9440 |   9328 |     13 |   2264 |    417 |  39073 |   1477 |  47476 |    333 |   5145 | 
logits:	 22.68 |   1.53 |   7.14 |   8.36 |   0.53 |   1.24 |  -1.31 |  -0.13 |   0.08 | -10.10 |  -5.05 |   2.72 | 
batch: 3
tokens:	   220 |    220 |    220 |    220 |    220 |    220 |    220 |    220 |    220 |    220 |  10910 |     13 | 
logits:	 22.68 |  18.83 |  18.52 |  18.08 |  19.06 |  20.29 |  21.18 |  21.54 |  21.77 |  21.78 |   1.73 |   0.93 | 


In [53]:
for i, a in enumerate(padded[:, :-1]):
    print(i, a)

0 [  220   220   220 10910    11   299   516  7608   346 24247   275]
1 [40932   502  3652  4763   271   288   504   443  1036   271 38078]
2 [  220  2538  9440  9328    13  2264   417 39073  1477 47476   333]
3 [  220   220   220   220   220   220   220   220   220   220 10910]


---

## Logits

In [32]:
def get_logits(model, context_tokens, last_only=False):
    """
    Generate the logits (probabilities of each token) at each step for a
    given one or more sequences of tokens.
    Returns:
    --------
        logits: array of shape: (batch_size, n_tokens, n_vocab)
                or, if last_only is True: (batch_size, 1, n_vocab)
    """
    # model.check_batch_size(batch_size)
    if not isinstance(context_tokens[0], (list, tuple, np.ndarray)):
        model.check_batch_size(1)
        context = model.batch_size * [context_tokens]
        print('one dim')
        print(context)
    else:
        model.check_batch_size(len(context_tokens))
        context = context_tokens
        print('many dims')        
        print(context)
    # model.model, defined at initialization, returns logits & attention matrix
    logits = model.sess.run(model.model, feed_dict={model.context: context})["logits"]
    # all logits starting from the second token, n logits for n tokens
    # shape (batch_size, n_tokens, vocab_size)
    if not last_only:
        return logits
    # logits for next token, None to keep dims intact
    return logits[:, None, -1, :]

In [35]:
get_logits(m, m.encode(snt[0])).shape

one dim
[array([10910,    11,   299,   516,  7608,   346, 24247,   275,  2013])]


(1, 9, 50257)

In [27]:
get_logits(m, m.pad_sequences(snt)).shape

many dims
[[  220   220   220 10910    11   299   516  7608   346 24247   275  2013]
 [40932   502  3652  4763   271   288   504   443  1036   271 38078 16245]
 [  220  2538  9440  9328    13  2264   417 39073  1477 47476   333  5145]
 [  220   220   220   220   220   220   220   220   220   220 10910    13]]


(4, 12, 50257)

In [71]:
get_logits(m, snt)[:, :-1, :].shape

(4, 11, 50257)

In [64]:
get_logits(m, snt, last_only=True).shape

(4, 1, 50257)

---

## Sequence padding

In [9]:
m.pad_sequences(['blah blah', 'blah diblah'])

array([[  220,   220,  2436,   993, 33367],
       [ 2436,   993,   288,   571,  9271]], dtype=int32)

In [16]:
tf.keras.preprocessing.sequence.pad_sequences(
    m.encode(snt), 
    maxlen=None, 
    dtype='int32', 
    padding='pre', 
    truncating='pre',
    value=220
)

array([[  220,   220,   220, 10910,    11,   299,   516,  7608,   346,
        24247,   275,  2013],
       [40932,   502,  3652,  4763,   271,   288,   504,   443,  1036,
          271, 38078, 16245],
       [  220,  2538,  9440,  9328,    13,  2264,   417, 39073,  1477,
        47476,   333,  5145],
       [  220,   220,   220,   220,   220,   220,   220,   220,   220,
          220, 10910,    13]], dtype=int32)

In [20]:
m.encode(' ')

array([220])

After having coded the function into the class (220 is space):

In [19]:
print(m.pad_sequences(snt, padding='pre'))
m.decode(m.pad_sequences(snt, padding='pre'))

[[  220   220   220 10910    11   299   516  7608   346 24247   275  2013]
 [40932   502  3652  4763   271   288   504   443  1036   271 38078 16245]
 [  220  2538  9440  9328    13  2264   417 39073  1477 47476   333  5145]
 [  220   220   220   220   220   220   220   220   220   220 10910    13]]


['   Ah, nous voilà bien',
 'Je me baladais dans le gris sans fond',
 ' LE COMTE. Quel déshonneur !',
 '          Ah.']

In [18]:
print(m.pad_sequences(snt, padding='post'))
m.decode(m.pad_sequences(snt, padding='post'))

[[10910    11   299   516  7608   346 24247   275  2013   220   220   220]
 [40932   502  3652  4763   271   288   504   443  1036   271 38078 16245]
 [ 2538  9440  9328    13  2264   417 39073  1477 47476   333  5145   220]
 [10910    13   220   220   220   220   220   220   220   220   220   220]]


['Ah, nous voilà bien   ',
 'Je me baladais dans le gris sans fond',
 'LE COMTE. Quel déshonneur ! ',
 'Ah.          ']

Encoding and decoding in turn seem to yield identity.

In [20]:
print(m.encode(m.decode(m.pad_sequences(snt, padding='post'))) == m.pad_sequences(snt, padding='post'))

[[ True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True]]
